In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install gpt4all

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [4]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "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"

In [6]:
from langchain_community.llms import GPT4All

llm = GPT4All(
    model=r"C:\Users\leamd\Documents\Cours\Atelier_Pratique_II\langchain\mistral-7b-openorca.gguf2.Q4_0.gguf",
    max_tokens=2048,
)

In [7]:
from langchain_community.embeddings import GPT4AllEmbeddings

In [8]:
#read txt file
with open("href_list.txt", "r") as file:
    text = file.read()

#split text into list of urls
web_urls = text.split("\n")

pdf_urls = []
for url in web_urls:
    if url == "":
        web_urls.remove(url)

    #if url is a pdf, add it to the pdf_urls list
    if url.endswith(".pdf") > 0:
        pdf_urls.append(url)
        web_urls.remove(url)
#remove empty strings
web_urls = list(filter(None, web_urls))

In [9]:
from langchain_community.document_loaders import PyPDFLoader

pages = []
for url in pdf_urls:
    loader = PyPDFLoader(url)
    pages += loader.load()

In [10]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=(web_urls),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("entry-header", "entry-content")
        )
    ),
)
docs = loader.load()
docs += pages

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=GPT4AllEmbeddings())

In [11]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(k=5)
# prompt = hub.pull("rlm/rag-prompt")

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [12]:
#rag_chain.invoke("Quel est la dernière date pour le conseil d'administation?")

In [13]:
from langchain_community.chat_message_histories import ChatMessageHistory

In [14]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

### Contextualize question ###
contextualize_q_system_prompt = """Compte tenu de l'historique des discussions et de la dernière question de l'utilisateur \
qui pourrait faire référence au contexte dans l'historique des discussions, formuler une question autonome \
qui peut être compris sans l'historique des discussions. Ne répondez PAS à la question, \
reformulez-le simplement si nécessaire et sinon renvoyez-le tel quel."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """Vous êtes assistant pour les tâches de réponses aux questions. \
Utilisez les éléments de contexte récupérés suivants pour répondre à la question. \
Si vous ne connaissez pas la réponse, dites simplement que vous ne la savez pas. \
Utilisez trois phrases maximum et gardez la réponse concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [15]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [16]:
conversational_rag_chain.invoke(
    {"input": "Quel est la définition de 'Docteur honoris causa'?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

Parent run 99038e14-0996-4356-9b60-50e91c4a42b6 not found for run 104f03bf-d372-44c8-8c20-63fc7383296e. Treating as a root run.


'\nAI: Le titre de docteur honoris causa est décerné à des personnes dont le mérite exceptionnel justifie un témoignage public d’appréciation et d’estime. Il s’adresse à des personnes extérieures à la communauté universitaire. Ces personnes éminentes ont, par leurs contributions, atteint des résultats hors du commun en plus de laisser leur marque dans un domaine précis. Ce titre permet de souligner la carrière, l’œuvre ou l’engagement d’une personne d’exception. La portée des réalisations ou du mérite de la personne se situe au niveau provincial, national ou international.'

In [17]:
conversational_rag_chain.invoke(
    {"input": "Et celle de « Ambassadrice, ambassadeur » dans le contexte de l'UQAC?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

Parent run 58d184a2-4788-4be5-87aa-a6e10de990ff not found for run 6b7d1b5b-735e-4101-a4dd-d76dee28a830. Treating as a root run.


"\nAI: L'Université du Québec à Chicoutimi (UQAC) peut décider d’attribuer un titre honorifique d’«Ambassadeur» ou «Ambassadrice». Ce titre est réservé aux personnes qui ont apporté une contribution exceptionnelle et significative au développement de l'institution. Cela peut inclure des réalisations dans le domaine académique, scientifique, culturel ou social, ainsi que d’autres domaines pertinents pour la mission et les valeurs de l'UQAC. Les ambassadeurs honoraires peuvent être des personnalités locales, régionales, nationales ou internationales qui ont un lien étroit avec l'Université du Québec à Chicoutimi et contribuent au rayonnement de cette institution."

In [18]:
conversational_rag_chain.invoke(
    {"input": "Quelles sont les critères de sélection des personnes à risques dans le cas de la biosécurité pour l'UQAC?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

Parent run f03374c3-79cc-420c-8285-aad773a06fb3 not found for run 2e09950a-5950-4fcb-a720-2871d6d5de72. Treating as a root run.


"\nAI: \nAI: Les critères de sélection pour les personnes à risque en matière de biosécurité au sein de l’Université du Québec à Chicoutimi (UQAC) sont basés sur plusieurs facteurs. Tout d'abord, il est nécessaire que la personne soit impliquée dans des activités de recherche ou d'enseignement qui impliquent le manipulation de matériel infectieux ou d'organismes comportant des risques biologiques. Ensuite, les niveaux de confinement appropriés doivent être déterminés pour chaque projet en fonction du niveau de risque associé et des exigences spécifiques liées à la biosécurité. Les personnes sélectionnées doivent également suivre une formation approfondie sur les pratiques de sécurité, l'équipement de protection individuelle (EPI) et les procédures d'urgence en matière de biosécurité. Enfin, elles ou ils doivent être en mesure de mettre en œuvre des protocoles de travail rigou"